In [0]:
df = spark.read\
        .format("delta")\
        .load("/Volumes/workspace/alura/c5/bronze")

In [0]:
df.printSchema()

In [0]:
from pyspark.sql import functions as F

In [0]:
df_zipped = df.withColumn(
    "zipped",
    F.arrays_zip(
        "anuncio.area_total",
        "anuncio.area_util",
        "anuncio.banheiros",
        "anuncio.quartos",
        "anuncio.suites",
        "anuncio.vaga",
        "anuncio.caracteristicas",
        "anuncio.valores"
    )
).withColumn("exploded", F.explode_outer("zipped"))

df_exploded = df_zipped.select(
    F.col("id").alias("id_externo"),          # id raiz
    F.col("anuncio.id").alias("id_anuncio"),  # id dentro do struct anuncio
    F.col("anuncio.andar"),
    F.col("anuncio.tipo_anuncio"),
    F.col("anuncio.tipo_unidade"),
    F.col("anuncio.tipo_uso"),

    F.col("anuncio.endereco.bairro").alias("bairro"),
    F.col("anuncio.endereco.cidade").alias("cidade"),
    F.col("anuncio.endereco.estado").alias("estado"),
    F.col("anuncio.endereco.cep").alias("cep"),
    F.col("anuncio.endereco.zona").alias("zona"),
    F.col("anuncio.endereco.pais").alias("pais"),
    F.col("anuncio.endereco.rua").alias("rua"),
    F.col("anuncio.endereco.latitude").alias("latitude"),
    F.col("anuncio.endereco.longitude").alias("longitude"),

    F.col("exploded.area_total").alias("area_total"),
    F.col("exploded.area_util").alias("area_util"),
    F.col("exploded.banheiros").alias("banheiros"),
    F.col("exploded.quartos").alias("quartos"),
    F.col("exploded.suites").alias("suites"),
    F.col("exploded.vaga").alias("vaga"),
    F.col("exploded.caracteristicas").alias("caracteristica"),

    F.col("exploded.valores.condominio").alias("condominio"),
    F.col("exploded.valores.iptu").alias("iptu"),
    F.col("exploded.valores.tipo").alias("tipo_valor"),
    F.col("exploded.valores.valor").alias("valor_valor")
)

In [0]:
df_detalhado_alura = df.select("anuncio.*", "anuncio.endereco.*")

In [0]:
df_exploded.write\
    .format("delta")\
    .mode("overwrite")\
    .save("/Volumes/workspace/alura/c5/silver/exploded")

df_detalhado_alura.write\
    .format("delta")\
    .mode("overwrite")\
    .save("/Volumes/workspace/alura/c5/silver/alura")